In [108]:
import pandas as pd
import numpy as np


import warnings
warnings.filterwarnings('ignore')


import matplotlib.pyplot as plt
%matplotlib inline


import seaborn as sns

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [109]:
customerdf = pd.read_csv(r"C:\Users\haiss\Desktop\Ironhack\lab-imbalanced-data\files_for_lab\customer_churn.csv", sep = ',')
customerdf

cols = []
for i in range(len(customerdf.columns)): 
    cols.append(customerdf.columns[i].lower().replace(' ', '_')) 
customerdf.columns = cols

customerdf

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [110]:
pd.set_option('display.max_columns', None)
customerdf.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [111]:
customerdf.groupby('churn').count()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
churn,,,,,,,,,,,,,,,,,,,,
No,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174
Yes,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869


In [112]:
nan_df = customerdf.isna().sum()
nan_df

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [113]:
type_df = customerdf.dtypes
type_df

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [114]:
numerical_df = customerdf.select_dtypes(include=['number'])
numerical_df

,seniorcitizen,tenure,monthlycharges
0,0,1,29.85
1,0,34,56.95
2,0,2,53.85
3,0,45,42.30
4,0,2,70.70
...,...,...,...
7038,0,24,84.80
7039,0,72,103.20
7040,0,11,29.60
7041,1,4,74.40


### We will try to predict variable Churn using a logistic regression on variables tenure, SeniorCitizen,MonthlyCharges.

In [115]:
customerdf.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [116]:
df_churn = customerdf.drop(['customerid', 'gender', 'partner', 'dependents', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod','totalcharges'], axis =1)
df_churn

,seniorcitizen,tenure,monthlycharges,churn
0,0,1,29.85,No
1,0,34,56.95,No
2,0,2,53.85,Yes
3,0,45,42.30,No
4,0,2,70.70,Yes
...,...,...,...,...
7038,0,24,84.80,No
7039,0,72,103.20,No
7040,0,11,29.60,No
7041,1,4,74.40,Yes


### Extract the independent variables and scale them.

In [117]:
# X-y split

y = df_churn['churn']
X = df_churn.drop(['churn'], axis = 1)
X

,seniorcitizen,tenure,monthlycharges
0,0,1,29.85
1,0,34,56.95
2,0,2,53.85
3,0,45,42.30
4,0,2,70.70
...,...,...,...
7038,0,24,84.80
7039,0,72,103.20
7040,0,11,29.60
7041,1,4,74.40


In [118]:
# Normalizing X numerical values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
normalized_X = scaler.fit_transform(X) 
normalized_X = pd.DataFrame(normalized_X, columns = X.columns)
normalized_X

,seniorcitizen,tenure,monthlycharges
0,0.0,0.013889,0.115423
1,0.0,0.472222,0.385075
2,0.0,0.027778,0.354229
3,0.0,0.625000,0.239303
4,0.0,0.027778,0.521891
...,...,...,...
7038,0.0,0.333333,0.662189
7039,0.0,1.000000,0.845274
7040,0.0,0.152778,0.112935
7041,1.0,0.055556,0.558706


### Build the logistic regression model.

In [119]:
X = df_churn.drop(['churn'], axis = 1)
y = df_churn['churn']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


LR = LogisticRegression()
LR.fit(X_train, y_train)

LR.score(X_test, y_test)

0.8055358410220014

In [120]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [121]:
predic =LR.predict(X_test)
predic

print("precision: ",precision_score(y_test,predic, pos_label='Yes'))
print("recall: ",recall_score(y_test,predic, pos_label='Yes'))
print("f1: ",f1_score(y_test,predic, pos_label='Yes'))
print(classification_report(y_test, predic))

precision:  0.6911196911196911
recall:  0.47989276139410186
f1:  0.5664556962025317
              precision    recall  f1-score   support

          No       0.83      0.92      0.87      1036
         Yes       0.69      0.48      0.57       373

    accuracy                           0.81      1409
   macro avg       0.76      0.70      0.72      1409
weighted avg       0.79      0.81      0.79      1409



### Even a simple model will give us more than 70% accuracy. Why?

In [122]:
# The data is imbalanced.

In [123]:
# to predict the probabilities instead of a class
predicted_probabilities = LR.predict_proba(X_test)[:,1]
predicted_probabilities


array([0.25772613, 0.039935  , 0.0189691 , ..., 0.11299457, 0.01617396,
       0.29747283])

In [137]:

custom_threshold = 0.5  
custom_predictions = (predicted_probabilities >= custom_threshold).astype(int)
custom_predictions

array([0, 0, 0, ..., 0, 0, 0])

In [138]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,predic)

array([[956,  80],
       [194, 179]], dtype=int64)

### Synthetic Minority Oversampling TEchnique (SMOTE) is an over sampling technique based on nearest neighbors that adds new points between existing points. Apply imblearn.over_sampling.SMOTE to the dataset. Build and evaluate the logistic regression model. Is it there any improvement?

In [139]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 0, sampling_strategy = 1)
X_train_SMOTE, y_train_SMOTE = sm.fit_resample(X_train, y_train)

train_smote = pd.concat([X_train_SMOTE, y_train_SMOTE], axis = 1)
train_smote

,seniorcitizen,tenure,monthlycharges,churn
0,0,21,64.850000,No
1,0,54,97.200000,No
2,0,1,23.450000,Yes
3,0,4,70.200000,Yes
4,0,0,61.900000,No
...,...,...,...,...
8271,0,7,95.361237,Yes
8272,0,4,80.963614,Yes
8273,0,18,88.318056,Yes
8274,1,3,83.754083,Yes


In [140]:
LR = LogisticRegression()
LR.fit(X_train_SMOTE, y_train_SMOTE)

# Indicatores

print("precision: ",precision_score(y_test,predic, pos_label='Yes'))
print("recall: ",recall_score(y_test,predic, pos_label='Yes'))
print("f1: ",f1_score(y_test,predic, pos_label='Yes'))
print(classification_report(y_test, predic))

precision:  0.6911196911196911
recall:  0.47989276139410186
f1:  0.5664556962025317
              precision    recall  f1-score   support

          No       0.83      0.92      0.87      1036
         Yes       0.69      0.48      0.57       373

    accuracy                           0.81      1409
   macro avg       0.76      0.70      0.72      1409
weighted avg       0.79      0.81      0.79      1409



In [141]:
confusion_matrix(y_test, predic)

array([[956,  80],
       [194, 179]], dtype=int64)